In [1]:
#!pip install simple_fpa

In [2]:
%%capture out
! cd ..; pip install .

In [3]:
print((out.stdout.split('\n')[-2]))

In [4]:
from simple_fpa import Model, load_haile
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.available
#plt.style.use('seaborn-white')

rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Arial"],
    "lines.linewidth": 1,
    "figure.dpi":200
})

rcParams['figure.figsize'] = 4, 2.5

In [5]:
data = load_haile()

In [6]:
specs = [[2], [3], range(2,6), range(5,10), range(2,10)]

In [7]:
for spec in specs:
    print('*'*30)
    print('specification: ', spec)

    model = Model(data = data, auctionid_columns = ['auctionid'], bid_column = 'actual_bid')
    model.data = model.data[model.data._bidders.isin(spec)].copy()

    print('observations:', len(model.data))

    cont_covs = ['adv_value', 'hhi', 'volume_total_1']
    disc_covs = ['year', 'forest']
    model.residualize(cont_covs, disc_covs, 'multiplicative')

    model.trim_residuals(5)
    model.fit(smoothing_rate = 0.33, trim_percent = 5, boundary = 'zero')
    model.predict()

    model.find_optimal_u()
    opt_idx = model.data._hat_rev.idxmax()
    opt_u = model.data._u[opt_idx]
    model.find_expected_fitted()
    print('bandwidth:', np.round(model.band,3))

    for confidence in [90,95]:
        draws = 1000

        model.make_cicb(confidence, draws = draws, hyp = 'onesided')
        
        leftmost = model.data[model.data._u == 0]._hat_rev.values
        highest_with_cb = np.max(model.data._hat_rev - model.data._rev_cb - leftmost)
        highest_with_ci = np.max(model.data._hat_rev[opt_idx] - model.data._rev_ci[opt_idx] - leftmost)
        
        print(confidence, ':: uniform :', np.round(highest_with_cb, 4), ': pointwise :',  np.round(highest_with_ci, 4))

******************************
specification:  [2]
observations: 10328
optimal exclusion: 0.09803
expected fitted value: 7236967.04
bandwidth: 0.01
90 :: uniform : 0.0241 : pointwise : 0.0248
95 :: uniform : 0.0241 : pointwise : 0.0246
******************************
specification:  [3]
observations: 12477
optimal exclusion: 0.1238
expected fitted value: 9678504.65
bandwidth: 0.009
90 :: uniform : 0.0233 : pointwise : 0.0249
95 :: uniform : 0.0232 : pointwise : 0.0246
******************************
specification:  range(2, 6)
observations: 43387
optimal exclusion: 0.12122
expected fitted value: 10659051.14
bandwidth: 0.006
90 :: uniform : 0.0242 : pointwise : 0.0251
95 :: uniform : 0.0241 : pointwise : 0.025
******************************
specification:  range(5, 10)
observations: 26841
optimal exclusion: 0.18352
expected fitted value: 16257097.94
bandwidth: 0.007
90 :: uniform : 0.0251 : pointwise : 0.028
95 :: uniform : 0.0248 : pointwise : 0.0275
******************************
specif